# Waves: A dive into PDEs

So far we have mostly dealt with ODEs. However, wave propagation is described with a partial differential equation (PDE). In general, it is much harder to solve PDEs than ODEs because a PDE has at least two independent variables whereas an ODE contains only one. We have systematic methods such as the Runge-Kutta scheme that are applicable to most initial value problems involving ODEs. For PDEs, however, few such widely applicable approaches exist. The correct method depends on the kind of problems, and the type of boundary conditions. So, we are going to look at different classes of problems.

Imagine a pulse propagating in a string. As the wave travels each segment of the string moves up and down perpendicular to the direction of propagation. At the macroscopic level, we observe a transverse wave that moves along the string, and the individual motion of the segments is not relevant. In contrast, at the microscopic level, we see discrete particles undergoing oscillatory motion perpendicular to the motion of the wave.

In a string, a solid, or a fluid, the collective motion of the particles determine the velocity of sound waves in the medium, as well as the thermal transport properties.

## Coupled oscillators: Warm-up to waves


![springs](figures/springs.jpg)


Let us first consider a one-dimensional chain of $N$ particles of mass $m$ with equal equilibrium separation $a$. The particles are coupled to massless springs with force constant $k_{c}$, except for the first and last springs at the two ends of the chain which have spring constant $k$. the individual displacement of the particle $i$ from its equilibrium position along the $x$ axis is called $u_{i}$. The ends of the fist and last spring are assumed fixed: $$u_{0}=u_{N+1}=0.$$ 
Since the force of an individual mass is determined only by the compression or expansion of the adjacent springs, the equation of motion for particle
$i$ is given by:
$$\begin{align}
m\frac{d^{2}u_{i}}{dt^{2}} =-k_{c}(u_{i}-u_{i+1})-k_{c}(u_{i}-u_{i-1}) \\
=-k_{c}(2u_{i}-u_{i+1}-u_{i-1}).\end{align}$$

The equations for particles $i=1$ and $i=N$ next to the walls are given by 
$$\begin{align}
m\frac{d^2u_1}{dt^2}=-k_c(u_1-u_2)-ku_1, \\
m\frac{d^2u_N}{dt^2}=-k_c(u_N-u_{N-1})-ku_N.\end{align}$$

Note that for $k_c=0$ all the equations will decouple and the motion of the particles become independent of their neighbors. The above equations describe longitudinal oscillations, *i. e.* motion along the direction of the chain. The equations for transverse motion are equivalent.

### Model the coupled oscillator
Let us attempt to model such an oscillator as described above.


In [ ]:
 # We recycle our 1D particle object
class particle(object): 
    
    def __init__(self, mass=1., x=0., v=0.):
        self.mass = mass
        self.f = 0
        self.x = x
        self.v = v
        
    def move(self, dt):
        # Euler-Cromer
        self.v = self.v + self.f/self.mass*dt
        self.x = self.x + self.v*dt

# Introduce a new class with some useful finite difference methods
class CoupledSystem(object):

    def __init__(self, N, k, kc, dt): 
        self.N = N
        self.dt = dt
        self.k = k
        self.kc = kc
        # we create a list of N particles
        self.particles = [particle() for _ in range(N)]
         
    def evolve(self):        
        # Compute the interaction forces with the other particles
        for i in range(self.N-1):
            self.particles[i].f = self.kc*(self.particles[i+1].x + self.particles[i-1].x - 2*self.particles[i].x)
        self.particles[0].f = self.kc*(self.particles[1].x - self.particles[0].x) - self.k*self.particles[0].x
        self.particles[self.N-1].f = self.kc*(self.particles[N-2].x - self.particles[N-1].x) - self.k*self.particles[N-1].x

        for p in self.particles:
            p.move(self.dt)
            
    def energy(self):
        # Compute kinetic and potential energy
        ke = sum(0.5 * p.mass * p.v ** 2 for p in self.particles)
        pe = sum(0.5 * self.k * self.particles[i].x ** 2 for i in [0, self.N - 1])  # End particles
        pe += sum(0.5 * self.kc * (self.particles[i].x - self.particles[i - 1].x) ** 2 for i in range(1, self.N))  # Coupling springs
        return (ke,pe)


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

N = 2
k = 1.0 #spring constant of end springs
kc = 0.8
dt = 0.001
tmax = 10
nsteps = int(tmax/dt)

Sys = CoupledSystem(N, k, kc, dt)
#initial displacement of 1st particle
Sys.particles[0].x = 0.5

# Set recording variables
t = np.linspace(0.,tmax,nsteps)
x = np.zeros(shape=(N,nsteps))
energys = np.zeros((2, nsteps))

for i in range(nsteps):
    for n in range(Sys.N):
        x[n,i] = Sys.particles[n].x
    energys[:,i] = Sys.energy()
    Sys.evolve()
plt.figure()
for i in range(N):
    plt.plot(t, x[i,:], lw=2, label = f'Particle {i+1}')
plt.legend()
plt.xlabel('time')
plt.ylabel('x (displacement)');
plt.show()

# Plot energy conservation
plt.figure()
plt.plot(t, energys[0, :], label='Kinetic Energy')
plt.plot(t, energys[1, :], label='Potential Energy')
plt.plot(t, energys[0, :] + energys[1, :], label='Total Energy', linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Energy')
plt.legend()
plt.show()

### Side Quest
Can we find the natural frequencies of such a system? We can calculate the natural frequencies by using the numpy FFT module. The frequency interval is given by 
$$\Delta \omega = \frac{1}{N_{steps}\Delta t}$$

In [ ]:
# Only programmed for two masses
#FFT analysis
#rfftfreq
#rfft 
w = np.fft.rfftfreq(nsteps,dt)
u0 = np.fft.rfft(x[0,])
u1 = np.fft.rfft(x[1,])

#w = np.fft.fftshift(w)  # We could shift the indices to center the frequencies at zero
#u0 = np.fft.fftshift(u0)
#u1 = np.fft.fftshift(u1)

plt.xlim(0,1);
plt.plot(w, abs(u0), color='green', ls='-', lw=3)
plt.xlim(0,1);
plt.plot(w, abs(u1), color='red', ls='-', lw=3)
plt.show()

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot

N = 2
k = 1.0
dt = 0.01
# Very large tmax to get a greater sampling from oscillations
tmax = 200
nsteps = int(tmax/dt)

for kc in [2., 1.,.8, .5, .2, .1, .05, .01,0]:
    print("Trying kc = ", kc)
    S = CoupledSystem(N, k, kc, dt)
    S.particles[0].x = 0.5
    S.particles[1].x = 0

    t = np.linspace(0.,tmax,nsteps)
    x = np.zeros(shape=(N,nsteps))

    for i in range(nsteps):
        for n in range(S.N):
            x[n,i] = S.particles[n].x
        S.evolve()

    pyplot.plot(t, x[0,], color='green', ls='-', lw=3)
    pyplot.plot(t, x[1,], color='red', ls='-', lw=3)

    pyplot.xlabel('time')
    pyplot.ylabel('x');
    pyplot.show()

    w = np.fft.rfftfreq(nsteps,dt)
    u0 = np.fft.rfft(x[0,])
    u1 = np.fft.rfft(x[1,])

    #w = np.fft.fftshift(w)  # We shift the indices to center the frequencies at zero
    #u0 = np.fft.fftshift(u0)
    #u1 = np.fft.fftshift(u1)

    pyplot.xlim(0,1);
    pyplot.plot(w, abs(u0), color='green', ls='-', lw=3)
    pyplot.xlim(0,1);
    pyplot.plot(w, abs(u1), color='red', ls='-', lw=3)
    pyplot.show()



In [ ]:
from scipy.signal import find_peaks
# Simulation parameters
N = 2  # Number of particles
k = 1.0  # Spring constant
dt = 0.01  # Time step
tmax = 200  # Total simulation time
nsteps = int(tmax / dt)  # Number of steps
kc_values = [1.0, 0.5, 0.1]  # Coupling constant(s)

# Run simulation for each kc value
for kc in kc_values:
    system = CoupledSystem(N, k, kc, dt)
    
    # Initial conditions
    system.particles[0].x = 0.5  # Excite first particle
    
    # Store time evolution
    x = np.zeros((N, nsteps))
    
    for i in range(nsteps):
        for n in range(N):
            x[n, i] = system.particles[n].x
        system.evolve()
    
    # Plot time evolution
    plt.figure(figsize=(10, 5))
    for n in range(N):
        plt.plot(np.linspace(0, tmax, nsteps), x[n], label=f'Particle {n}')
    plt.xlabel('Time')
    plt.ylabel('Displacement')
    plt.legend()
    plt.title(f'Time Evolution (kc = {kc})')
    plt.show()
    
    # Compute FFT
    w = np.fft.rfftfreq(nsteps, dt)
    fft_magnitudes = []
    
    plt.figure(figsize=(10, 5))
    for n in range(N):
        u = np.fft.rfft(x[n])
        abs_u = abs(u)
        plt.plot(w, abs_u, label=f'Particle {n}')
        fft_magnitudes.append(abs_u)
    
    plt.xlabel('Frequency')
    plt.ylabel('Magnitude')
    plt.xlim(0, 1)
    plt.legend()
    plt.title(f'FFT Spectrum (kc = {kc})')
    plt.show()
    
    # Extract dominant frequencies
    fft_results = {}
    for n in range(N):
        peaks, _ = find_peaks(fft_magnitudes[n], height=0.01)
        peak_freqs = w[peaks]
        fft_results[f'Particle {n}'] = peak_freqs[:N]  # Store first two peaks
    
    # Print results
    print(f'FFT Peaks for kc = {kc}:')
    for key, freqs in fft_results.items():
        print(f'{key}: {freqs}')

# Waves on a string


![string](figures/string.jpg)
#### A stretched string of length $l$ with the ends fixed

The difference between waves and oscillatory motion is the scale. Waves are the “continuum” limit of the problem, or in the jargon “the long wavelength” limit. This is because in this limit, all the microscopic details are “washed out” and only the long distance behavior survives. In order to understand the transition between these two limits we have to perform a change of scale. The discrete equations of motion, as shown previously, can be written as:
$$\frac{d^{2}u}{dt^{2}}=-\frac{k}{m}(2u_{i}-u_{i+1}-u_{i-1}).$$ 
We consider the limits $$N\rightarrow \infty ,a\rightarrow 0.$$ with the length of the chain kept constant. The main result is that in this limit, the discrete equations of motion can me replaced by the continuous wave equation:
$$\frac{\partial ^{2}u(x,t)}{\partial t^{2}}={v^{2}}\frac{\partial
^{2}u(x,t)}{\partial x^{2}},  $$ 
where $v$ has the dimension of velocity and it is given by $$v=\sqrt{k/\rho },$$ where $k$ is the string tension and $\rho $ is the linear density. Observe that the displacement $u$ of the string is the dependent variable, and that the position along the string $x$ and the time $t$ are the independent variables. The existence of two independent variables makes this a Partial Differential Equation (PDE).

There are many solutions to this equation. Examples are:

$$\begin{align}
u(x,t) &=&A\cos \frac{2\pi }{\lambda }(x\pm vt), \\
u(x,t) &=&A\sin \frac{2\pi }{\lambda }(x\pm vt).\end{align}$$ 
In fact, it is easy to show that any function of the form $f(x\pm vt)$ is a solution. Since the differential equation is a linear equation and hence
satisfies the superposition principle, we can understand the behavior of a wave of arbitrary shape using the Fourier’s theorem to represent its shape as a sum of sinusoidal waves.

Because both ends of the string are tied down, the boundary conditions are: $$u(0,t)=u(l,t)=0.\,\,\,\mathrm{(boundary\,\,condition)}$$

Since this is a second order PDE, we still need to determine the initial distortion $u(x,t=0)$ and velocity $\partial u/\partial t(x,t=0)$. If the string is released from rest, this reduces to 
$$\begin{align} u(x,t) =f(x),\,\,\,\,\mathrm{(initial\,\,condition\,\,1)}\\
\frac{\partial u}{\partial t}(x,t =0)=0.\,\,\,\mathrm{(initial\,\,condition\,\,2)}  \end{align}$$

## Numerical solution: Finite Differences Method (FDM)

![differences](figures/differences.jpg)
#### Finite differences grid for the vibrating string

To solve the the wave equation as a function of position and time we need to discretize the $(x,t)$ space in a rectangular grid. In the present case, the horizontal axis represents the position $x$ along the string, and the vertical axis represent time. We convert the equation to a finite difference equation expressing the second derivatives in terms of differences

$$\begin{align}
\frac{\partial ^{2}u(x,t)}{\partial t^{2}} &\simeq &\frac{u(x,t+\Delta t)+u(x,t-\Delta t)-2u(x,t)}{(\Delta t)^{2}}, \\
\frac{\partial ^{2}u(x,t)}{\partial x^{2}} &\simeq &\frac{u(x+\Delta x,t)+u(x-\Delta x,t)-2u(x,t)}{(\Delta x)^{2}}.\end{align}$$

After substituting into the wave equation, we obtain the discrete equations:
$$u(x,t+\Delta t)=2u(x,t)-u(x,t-\Delta t)+\frac{v^{2}}{C^{2}}\left[ u(x+\Delta x,t)+u(x-\Delta x,t)-2u(x,t)\right] ,$$
with $C=\Delta x/\Delta t$ is a constant with the dimension of velocity.

As shown in the figure above, this is a recurrence relation that propagates the wave from the two earlier times $t-\Delta t$ and $t$, and the three nearby positions $x-\Delta x$, $x$, and $x+\Delta x$, to a later time $t+\Delta t$, and a single position $x$. We can see right a way that this is not a self starting algorithm, in the sense that we need to know the position for two earlier times to start the iteration. However, we can use a simple trick to overcome this difficulty. Rewriting the initial conditions from above into the finite differences form, we obtain: 
$$\begin{align}
\frac{\partial u(x,t=0)}{\partial t} =0&\Rightarrow& \frac{u(x,\Delta t)-u(x,-\Delta t)}{2\Delta t}=0, \\
&\Rightarrow &u(x,-\Delta t)=u(x,\Delta t).\end{align}$$ 
Using this condition for the first iteration we obtain
$$u(x,t+\Delta t)=u(x,0)+\frac{1}{2}\frac{v^2}{C^2}\left[ u(x+\Delta x,0)+u(x-\Delta x,0)-2u(x,0)\right] .$$ 



## CFL Stability Criterion
The success of this method depends on the relative sizes of the time and space steps. The Courant-Friedrichs-Lewy condition is a necessary condition for convergence while solving certain partial differential equations numerically. This stability criterion says that the finite difference algorithm is stable if $$v\leq \frac{\Delta x}{\Delta t}=C.$$ This means that the solution gets better with smaller time steps, but worse with smaller space steps!

## **Why would this be?**


# Simulations
Let's use FDM to simulate waves on a 1D string. To do so, we need to check the CFL stability conditions and then use the discretized wave equations to propagate an answer across the string.

## First Attempt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set up the spatial and temporal discretization
L = 1.0  # Length of string
Nx = 100  # Number of spatial points
dx = L / (Nx - 1)  # Spatial step
c = 1.0  # Wave speed
dt = 0.005  # Time step
Nt = 500  # Number of time steps

# CFL condition check
assert c <= dx/dt, "CFL condition violated! Reduce dt or increase dx."

# Initialize displacement arrays
u = np.zeros(Nx)  # Current state
u_prev = np.zeros(Nx)  # Previous state
u_next = np.zeros(Nx)  # Next state

# Initial condition: A Gaussian pulse in the middle
x = np.linspace(0, L, Nx)
u[:] = np.exp(-100 * (x - 0.5) ** 2)
u_prev[:] = u[:]

# Set up the figure
fig, ax = plt.subplots()
ax.set_xlim(0, L)
ax.set_ylim(-1, 1)
line, = ax.plot([], [], lw=2)

# Update function for animation
def update(frame):
    global u, u_prev, u_next
    
    # Apply finite difference scheme (2nd order central difference in time & space)
    for i in range(1, Nx - 1):
        u_next[i] = 2 * u[i] - u_prev[i] + (c * dt / dx) ** 2 * (u[i+1] - 2 * u[i] + u[i-1])
    
    # Update arrays
    u_prev[:], u[:] = u[:], u_next[:]
    
    # Update the line in the plot
    line.set_data(x, u)
    return line,

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=Nt, interval=20, blit=True)

# Embed animation as HTML
HTML(ani.to_jshtml())


## Second attempt: Trying to store information in arrays

In [ ]:
# ***This Code suffers from something, but I cannot seem to find what**
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set up the spatial and temporal discretization
L = 2.0  # Length of string
Nx = 100  # Number of spatial points
dx = L / (Nx - 1)  # Spatial step
c = 1.0  # Wave speed
dt = 0.005  # Time step
Nt = 500  # Number of time steps

# CFL condition check
assert c * dt / dx <= 0.5, "CFL condition violated! Reduce dt or increase dx."

# Initialize displacement array
u = np.zeros((Nt, Nx))  # Store all time steps

# Initial condition: A Gaussian pulse in the middle
x = np.linspace(0, L, Nx)
u[0, :] = np.exp(-100 * (x - 0.5) ** 2)
# Alternative initial conditions:
# Sharp pulse: 
#u[0, :] = np.where((x > 0.45) & (x < 0.55), 1, 0)
# Triangular wave: 
#u[0, :] = 0.5*np.maximum(0, 1 - np.abs(x - 0.5))
u[1, :] = u[0, :].copy()  # First time step (assumes initial velocity is zero)

# Set up the figure
fig, ax = plt.subplots()
ax.set_xlim(0, L)
ax.set_ylim(-1, 1)
line, = ax.plot([], [], lw=2)

# Update function for animation
def update(frame):
    # Apply finite difference scheme (2nd order central difference in time & space)
    if frame > 0:
        u[frame, 1:-1] = 2 * u[frame - 1, 1:-1] - u[frame - 2, 1:-1] + (c * dt / dx) ** 2 * (u[frame - 1, 2:] - 2 * u[frame - 1, 1:-1] + u[frame - 1, :-2])
    
    # Update the line in the plot
    line.set_data(x, u[frame, :])
    return line,

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=Nt, interval=20, blit=True)

# Embed animation as HTML
HTML(ani.to_jshtml())

## Third Attempt: Trying to fix some stability issues
After looking through the above code, some changes needed to be made to ensure that the solution didn't run away from us. There is also a concern about the amound of memory used. The previous and current parameters in the update function try to alleviate that by storing only a limited number of time points.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set up the spatial and temporal discretization
L = 1.0  # Length of string
Nx = 100  # Number of spatial points
dx = L / (Nx - 1)  # Spatial step
c = 1.0  # Wave speed
dt = 0.005  # Time step
Nt = 500  # Number of time steps

# CFL condition check
assert c <= dx/dt, "CFL condition violated! Reduce dt or increase dx."

# Initialize displacement array (2 rows: one for current, one for previous state)
u = np.zeros((2, Nx))  

# Initial condition: A Gaussian pulse in the middle
x = np.linspace(0, L, Nx)
u[0, :] = np.exp(-100 * (x - 0.5) ** 2)  # Initial displacement
u[1, :] = u[0, :]  # Same for previous state

# Set up the figure
fig, ax = plt.subplots()
ax.set_xlim(0, L)
ax.set_ylim(-1, 1)
line, = ax.plot([], [], lw=2)

# Update function for animation
def update(frame):
    global u
    
    previous = frame % 2   # Alternating index: 0 or 1
    current = (frame + 1) % 2  # Next state index

    # Apply finite difference scheme
    u[current, 1:-1] = 2 * u[previous, 1:-1] - u[current, 1:-1] + (c * dt / dx) ** 2 * (
        u[previous, 2:] - 2 * u[previous, 1:-1] + u[previous, :-2]
    )

    # Update the line in the plot
    line.set_data(x, u[current, :])
    return line,

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=Nt, interval=20, blit=True)

# Embed animation as HTML
HTML(ani.to_jshtml())

## A 2D Wave
What must change for the simulation to be a wave on a 2D sheet instead of a 1D string?